In [1]:
#import necessary libraries
import requests
import pandas as pd
import csv
import json
import numpy as np

In [2]:
#To join data frames
def custom_join(left_df,right_df,join_type,left_col,right_on):
    temp=pd.merge(left_df,right_df,how=join_type,left_on=left_col,right_on=right_on)
    return temp

In [3]:
#variable declaration
assest_cat_li=["categories.json","asset-types.json","elements.json","attributes.json","picklists.json"]
main_url="https://environment.data.gov.uk/asset-management/drl-app/revision/current/api/list/"

sub_url="https://environment.data.gov.uk/asset-management/drl-app/revision/current/api/resource.json?id=http://environment.data.gov.uk/asset-management/id/drl/"

In [ ]:
"""
first getting and displaying all values under specific value
then we select a value from the options displayed
then it will move to next topic with the selected value as filter
this works from Category to Picklists (We can end at anytime but upto selected value will be displayed.)
"""

In [6]:
count=0
flag=True
main_df=pd.DataFrame()
id_val=""

while flag:
    if count==0:
        cat_json = requests.get(main_url+assest_cat_li[count]).json()
        cat_df=pd.json_normalize(cat_json)
        cat_val_li=[]
        #for split and store
        for val in cat_df['id'].values.tolist():
            cat_val_li.append(val.split("/")[-1])  
        print("Select Categories from listed below:")
        #for display the list
        for ind,val in enumerate(cat_val_li):
            print("\n",ind+1," - ",val)
        #getting an input from displayed options
        inp_val=int(input("\nEnter the index value: "))
        val_json=requests.get(sub_url+cat_val_li[inp_val-1]).json()
        main_df=pd.json_normalize(val_json)
        #reference column for upcoming jsons
        id_val=main_df['id'].loc[0]
    elif count==1:
        ast_json = requests.get(main_url+assest_cat_li[count]).json()
        ast_df=pd.json_normalize(ast_json,record_path=['markdown'],meta=['description','category','assetCode','label','id','uniclass2015',['edit','id'],['edit','type'],['edit','date']],errors='ignore',meta_prefix='meta-',record_prefix='record-')
        fil_df=ast_df[ast_df['meta-category']==id_val]
        
        ast_val_li=[]
        for val in set(fil_df['meta-id'].values.tolist()):
            ast_val_li.append(val.split("/")[-1]) 
        ast_val_li=list(set(ast_val_li))
        asset_sub_type_count=len(ast_val_li)
        print("Asset subtype count: ",asset_sub_type_count)
        print("Select Asset Type from listed below:")
        for ind,val in enumerate(ast_val_li):
            print("\n",ind+1," - ",val)
        inp_val=int(input("\nEnter the index value: "))
        val_json=requests.get(sub_url+ast_val_li[inp_val-1]).json()
        #print(val_json)
        ast_df=pd.json_normalize(val_json,record_path=['markdown'],meta=['description','category','assetCode','label','id','uniclass2015',['edit','id'],['edit','type'],['edit','date']],errors='ignore',meta_prefix='meta-',record_prefix='record-')
        main_df=custom_join(main_df,ast_df,"left","id","meta-category")
        id_val=main_df['meta-id'].loc[0]
    elif count==2:
        ast_json = requests.get(main_url+assest_cat_li[count]).json()
        ast_df=pd.json_normalize(ast_json)
        fil_df=ast_df[ast_df['assetType']==id_val]
        element_count=len(fil_df)
        print("Elements count: ",element_count)
        ast_val_li=[]
        for val in set(fil_df['id'].values.tolist()):
            ast_val_li.append(val.split("/")[-1])  
        print("Select Elements from listed below:")
        for ind,val in enumerate(ast_val_li):
            print("\n",ind+1," - ",val)
        inp_val=int(input("\nEnter the index value: "))
        val_json=requests.get(sub_url+ast_val_li[inp_val-1]).json()
        ast_df=pd.json_normalize(val_json)
        main_df=custom_join(main_df,ast_df,"left","meta-id","assetType")
        id_val=main_df['id_y'].loc[0]
    elif count==3:
        ast_json = requests.get(main_url+assest_cat_li[count]).json()
        ast_df=pd.json_normalize(ast_json)
        fil_df=ast_df[ast_df['attributeOf']==id_val]
        attribute_count=len(fil_df)
        print("Attributes count: ",attribute_count)
        ast_val_li=[]
        for val in set(fil_df['id'].values.tolist()):
            ast_val_li.append(val.split("/")[-1])  
        print("Select Attributes from listed below:")
        for ind,val in enumerate(ast_val_li):
            print("\n",ind+1," - ",val)
        inp_val=int(input("\nEnter the index value: "))
        val_json=requests.get(sub_url+ast_val_li[inp_val-1]).json()
        ast_df=pd.json_normalize(val_json)
        main_df=custom_join(main_df,ast_df,"left","id_y","attributeOf")
        id_val=main_df['format.id'].loc[0]
    elif count==4:
        ast_json = requests.get(main_url+assest_cat_li[count]).json()
        ast_df=pd.json_normalize(ast_json,record_path=['options'],meta=['id','label',['edit','id'],['edit','type'],['edit','date']],errors='ignore',meta_prefix='meta-',record_prefix='record-')
        fil_df=ast_df[ast_df['meta-id']==id_val]
        ast_val_li=[]
        for val in set(fil_df['meta-id'].values.tolist()):
            ast_val_li.append(val.split("/")[-1])  
        print("Select Picklists from listed below:")
        for ind,val in enumerate(ast_val_li):
            print("\n",ind+1," - ",val)
        inp_val=int(input("\nEnter the index value: "))
        val_json=requests.get(sub_url+ast_val_li[inp_val-1]).json()
        ast_df=pd.json_normalize(val_json,record_path=['options'],meta=['id','label',['edit','id'],['edit','type'],['edit','date']],errors='ignore',meta_prefix='meta-',record_prefix='record-')
        main_df=custom_join(main_df,ast_df,"left","format.id","meta-id")
        
    print("\nDo you want to continue (Y/N) ")
    ch_val=input()
    if ch_val=="Y":
        count+=1
        if count>4:
            flag=False
    else:
        flag=False

Select Categories from listed below:

 1  -  AidsToNavigation

 2  -  Amenities

 3  -  AssetComplex

 4  -  BeachStructure

 5  -  BuildingsAndCompounds

 6  -  Channel

 7  -  ChannelCrossings

 8  -  Defence

 9  -  InformationDeliverables

 10  -  Instruments

 11  -  Land

 12  -  Meica

 13  -  PublicSafety

 14  -  Structure

Enter the index value: 5

Do you want to continue (Y/N) 
Y
Asset subtype count:  15
Select Asset Type from listed below:

 1  -  VisitorCentre

 2  -  Garage

 3  -  MonitoringBuilding

 4  -  MaintenanceBuilding

 5  -  PumpHouse

 6  -  ResidentialBuilding

 7  -  GaugingStationBuilding

 8  -  Boathouse

 9  -  Shelter

 10  -  ControlBuilding

 11  -  OfficeBuilding

 12  -  StorageLocation

 13  -  Watermill

 14  -  AmenityBuilding

 15  -  LabBuilding

Enter the index value: 5

Do you want to continue (Y/N) 
Y
Elements count:  74
Select Elements from listed below:

 1  -  PumpHouse-frame

 2  -  PumpHouse-thrust-pit

 3  -  PumpHouse-alternator

 4  

ValueError: invalid literal for int() with base 10: 'N'

In [5]:
#drop the columns if needed
Final_Data=main_df.drop(['id_x','description_x', 'edit.id_x', 'edit.type_x',
       'edit.date_x', 'record-priority', 'record-desc', 'record-type',
       'record-image', 'record-altText', 'record-caption', 'meta-description',
       'meta-category', 'meta-assetCode', 'meta-id_x',
       'meta-uniclass2015', 'meta-edit.id_x', 'meta-edit.type_x',
       'meta-edit.date_x', 'description_y', 'assetType', 'elementCode',
       'id_y', 'essential','uniclass2015', 'edit.id_y',
       'edit.type_y', 'edit.date_y', 'description', 'mandatory', 'purpose',
       'geocobieMapping', 'delivery', 'id',
       'format.format', 
       'format.formatLabel', 'edit.id', 'edit.type', 'edit.date',
       'record-id', 'record-sortPriority', 'meta-id_y',
       'meta-edit.id_y', 'meta-edit.type_y',
       'meta-edit.date_y'],
      axis=1)

In [9]:
Final_Data.rename(columns = {"label_x":"Category-label","meta-label_x":"Asset subtypes-label","label_y":"Elements-label","label":"Attributes-label","meta-label_y":"Picklists-label"},inplace=True)

In [10]:
Final_Data.to_csv('Final_Data.csv',index=False)

In [14]:
main_df.columns.tolist()

['id_x',
 'label_x',
 'description_x',
 'edit.id_x',
 'edit.type_x',
 'edit.date_x',
 'record-priority',
 'record-desc',
 'record-type',
 'record-image',
 'record-altText',
 'record-caption',
 'meta-description',
 'meta-category',
 'meta-assetCode',
 'meta-label_x',
 'meta-id_x',
 'meta-uniclass2015',
 'meta-edit.id_x',
 'meta-edit.type_x',
 'meta-edit.date_x',
 'description_y',
 'assetType',
 'elementCode',
 'label_y',
 'id_y',
 'essential',
 'uniclass2015',
 'edit.id_y',
 'edit.type_y',
 'edit.date_y',
 'description',
 'mandatory',
 'purpose',
 'attributeOf',
 'geocobieMapping',
 'delivery',
 'label',
 'id',
 'format.id',
 'format.format',
 'format.formatLabel',
 'edit.id',
 'edit.type',
 'edit.date',
 'record-id',
 'record-label',
 'record-sortPriority',
 'meta-id_y',
 'meta-label_y',
 'meta-edit.id_y',
 'meta-edit.type_y',
 'meta-edit.date_y']

In [11]:
def check_val_in_col(val,df):
    res=[]
    for col in df.columns.tolist():
        if val in df[col].astype(str).values.tolist():
            res.append(col)
    return res

In [12]:
#ast_json = requests.get(main_url+"picklists.json").json()
#ast_df=pd.json_normalize(ast_json,record_path=['options'],meta=['id','label',['edit','id'],['edit','type'],['edit','date']],errors='ignore',meta_prefix='meta-',record_prefix='record-')
#print(check_val_in_col(id_val,ast_df))

['meta-id']


In [36]:
#for i in temp.columns.tolist():
    #print(i,check_val_in_col(temp[i].iloc[0],ast_df))

description []
mandatory []
purpose ['record-label']
attributeOf []
geocobieMapping []
delivery []
label []
id []
format.id []
format.format []
format.maxValue []
format.numberOfDecimalPlaces []
format.formatLabel []
format.minValue []
format.numericUnit []
edit.id []
edit.type ['meta-edit.type']
edit.date ['meta-edit.date']
format.maxLength ['record-sortPriority']
format.referenceType []
format.referenceCode []


In [21]:
temp['id'].iloc[0]

'http://environment.data.gov.uk/asset-management/id/drl/Groyne-fishtail-element-description'

In [43]:
temp['attributeOf'].iloc[0]

'http://environment.data.gov.uk/asset-management/id/drl/Groyne-fishtail'

In [14]:
id_val

'http://environment.data.gov.uk/asset-management/id/drl/Groyne-fishtail'

In [ ]:
main_df['meta-id'].values

In [ ]:
####jgaajha 